In [2]:
# Import standard libraries
import types
import os
import sys

# Import third-party libraries
import numpy as np
import pandas as pd

In [2]:
#### Test area equality constraints (Ax = b) ####

n = 4
A_aug = np.array([1,1,1,1,0,0,0,0])
#A_aug = None
A_add_con_1 = np.zeros(2 * n)
A_add_con_2 = np.zeros(2 * n)
A_add_con_3 = np.zeros(2 * n)
A_add_con_1[n + 0] = 1
A_add_con_2[n + 1] = 1
A_add_con_3[n + 2] = 1

# Take a snapshot of globals so it doesn't change during iteration
global_vars = dict(globals())
A_add_con_list = []

for name, con in global_vars.items():
    if name.startswith("A_add_con_") and isinstance(con, np.ndarray):
        if not np.all(con == 0):
            A_add_con_list.append(con)

# Stack them if any valid constraints exist
A_add_cons = np.vstack(A_add_con_list) if A_add_con_list else None

if A_add_cons is not None:
    if A_aug is not None:
        A_aug = np.vstack([A_aug, A_add_cons])
    else:
     A_aug = A_add_cons.reshape(len(A_add_cons), -1)

b = np.array([1])
#b = None

if A_add_cons is not None:
    b_add_cons = np.zeros(len(A_add_cons))
    b_add_cons[0] = 0.3
    #b_add_cons[1] = 0.5
    #b_add_cons[2] = 2
    if b is not None:
        b_aug = np.hstack([b, b_add_cons])
    else:
        b_aug = A_add_cons.reshape(len(A_add_cons), -1)
else:
    b_aug = b

print(b_aug)
print(A_aug)
#print(len(A_aug))
#print(len(b_aug))

[1.  0.3 0.  0. ]
[[1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]]


In [11]:
#### Test area inequality constraints (Gx <= h) ####

n = 2
#G = np.ones((n, n))
G = None
G_add_con_list = []

if G is not None:
    G_aug = np.pad(G, ((0, 0), (0, n)), mode='constant')
    for i in range(n):
        row = np.zeros(2 * n)
        row[i] = 1
        row[i + n] = -1
        G_add_con_list.append(row)
        neg_row = row.copy()
        neg_row[i] = -1
        neg_row[i + n] = -1
        G_add_con_list.append(neg_row)
    row_tau = np.hstack([np.zeros(n), np.ones(n)])
    G_aug = np.vstack([G_aug, G_add_con_list, row_tau])
else:
    for i in range(n):
        row = np.zeros(2 * n)
        row[i] = 1
        row[i + n] = -1
        G_add_con_list.append(row)
        neg_row = row.copy()
        neg_row[i] = -1
        neg_row[i + n] = -1
        G_add_con_list.append(neg_row)
    row_tau = np.hstack([np.zeros(n), np.ones(n)])
    G_aug = np.vstack([G_add_con_list, row_tau])

#h = np.ones(n)
h = None
x_init = np.array([0.7, 0.3])
to_budget = 0.5

if h is not None:
    x_init_adj = np.empty((2 * len(x_init),), dtype=float)
    x_init_adj[0::2] = x_init # Start at index 0, go to the end, and take every 2nd element and assign "x_init" to it.
    x_init_adj[1::2] = -x_init # Start at index 1, go to the end, and take every 2nd element and assign "-x_init" to it.
    h_aug = np.hstack([h, x_init_adj, to_budget])
else:
    x_init_adj = np.empty((2 * len(x_init),), dtype=float)
    x_init_adj[0::2] = x_init
    x_init_adj[1::2] = -x_init
    h_aug = np.hstack([x_init_adj, to_budget])

print(G_aug)
print(h_aug)

[[ 1.  0. -1.  0.]
 [-1.  0. -1.  0.]
 [ 0.  1.  0. -1.]
 [ 0. -1.  0. -1.]
 [ 0.  0.  1.  1.]]
[ 0.7 -0.7  0.3 -0.3  0.5]
